# Hourly Staff Planning

In [1]:
import numpy as np
import pandas as pd

Staff planning is a topic of optimization research that comes back in many companies. As soon as a company has many employees, it becomes hard to find planning that suits the business needs while respecting certain constraints.

I'll be starting with the shape of the staff planning that is required. Then, I'll use genetic algorithms to predict an optimal staff planning strategy in which all employees work every weekday, 5 days a week (the shop is closed on the weekend). I'll be working from these assumptions:

* A shift is given by a starting time and shift duration
* I have data on the number of staff needed per hour
* An employee can be planned to not work on a certain day

## Staff Planning Initial Shape
The staff planning is represented as a list per day. There are 5 lists for each of the 5 days
Each day consists of many lists of lenght 3
Each list of 3 is an employee with the following items: (staff id, starting time, shift duration)
The number of lists is the number of employees that are possibly available on that day

In [2]:
staff_planning = [
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]],
    [[0, 0, 10],[1, 0, 10],[2, 0, 10],[3, 0, 10],[4, 0, 10],[5, 0, 10],[6, 0, 10],[7, 0, 10],[8, 0, 10],[9, 0, 10],[10, 0, 10]]
]

In [3]:
staff_planning[0][3]

[3, 0, 10]

## Staff Planning for Shop
In order to optimize the staff planning, I need information on what would be perfect planning
Based on previous days, I know how many staff are needed every hour
The staff needed is in the following shape:
<li> list of days </li>
<li> each day is a list of 24 hours, with the number of employees needed every hour </li>

In [4]:
hourlystaff_needed = np.array([
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2, 2, 2]
])

In [5]:
hourlystaff_needed[0]

array([0, 0, 0, 0, 0, 0, 4, 4, 4, 2, 2, 2, 6, 6, 2, 2, 2, 6, 6, 6, 2, 2,
       2, 2])

## Conversion from shifts to staff-per-hour
<li> In the optimization, the genetic algorithm will iteratively change the starting times and the durations </li>
<li> This is then converted into number of employees per hour </li>
<li> Then I measure how far away this is from the staff-needed planning </li>

In order to do this, I will need a function to convert one type of planning into the other one

In [6]:
# analyze whether the employee is present at a given time with yes/no
# based on the employee list of 3 (id, start time, duration)
def employee_present(employee, time):
    employee_start_time = employee[1]
    employee_duration = employee[2]
    employee_end_time = employee_start_time + employee_duration
    if (time >= employee_start_time) and (time < employee_end_time):
        return True
    return False

In [7]:
employee_present(staff_planning[0][3], 9)

True

In [8]:
# convert the staff planning (lists of staff id, starting time, shift duration) to the staff present at each hour
def staffplanning_to_hourlyplanning(staff_planning):
    hourlystaff_week = []
    for day in staff_planning:
        
        hourlystaff_day = []
        for employee in day:
            
            employee_present_hour = []
            for time in range(0, 24):
                # an array for each employee and True/False for whether they are working each hour
                employee_present_hour.append(employee_present(employee, time))
            
            # an array of all employees working in a day and True/False for each hour
            hourlystaff_day.append(employee_present_hour)
        
        # an array containing a week's worth of data of all employees working each day and True/False for each huor
        hourlystaff_week.append(hourlystaff_day)
    
    # counts up the number of staff working each hour of each day
    hourlystaff_week = np.array(hourlystaff_week).sum(axis = 1)
    return hourlystaff_week

In [9]:
hourlystaff_week = staffplanning_to_hourlyplanning(staff_planning)

In [10]:
hourlystaff_week

array([[11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0],
       [11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0]])

## Cost function to evaluate how well the staff planning performed based on the staff needed

In [11]:
# the cost is calculated as hours understaffed + hours overstaffed
def cost(hourlystaff, hourlystaff_needed):
    # finds the difference between the actual hourly staff compared with the hourlystaff needed
    errors = hourlystaff - hourlystaff_needed
    
    # calculates how many hours were overstaffed and by how much
    overstaff = abs(errors[errors > 0].sum())
    
    # calculates how many hours were understaffed and by how much
    understaff = abs(errors[errors < 0].sum())
    
    # assigning a cost to each hour overstaffed or understaffed
    overstaff_cost = 1
    understaff_cost = 1
    
    # calculating the cost function
    cost = overstaff_cost * overstaff + understaff_cost * understaff
    return cost

In [12]:
# cost of default
cost(hourlystaff_week, hourlystaff_needed)

720

## Random initialization
First, I will need to randomly initialize the planning. There are a fixed number of days and staff. The start time of each employee of each day is a random choice between 0-23 and the duration of each employee of each day is a random choice between 0-10

In [13]:
# generate an entirely random staff planning for a certain number of staff and days
def generate_random_staff_planning(n_days, n_staff):
    period_planning = []
    for day in range(n_days):
        day_planning = []
        for employee_id in range(n_staff):
            start_time = np.random.randint(0, 23)
            duration = np.random.randint(0, 10)
            employee = [employee_id, start_time, duration]
            day_planning.append(employee)
        period_planning.append(day_planning)
    return period_planning

In [14]:
# sample random week with 5 days and 3 employees
sample_random_week = generate_random_staff_planning(5, 3)
sample_random_week

[[[0, 5, 9], [1, 11, 6], [2, 9, 9]],
 [[0, 20, 0], [1, 9, 5], [2, 6, 0]],
 [[0, 10, 8], [1, 10, 0], [2, 21, 8]],
 [[0, 20, 3], [1, 0, 2], [2, 13, 0]],
 [[0, 21, 0], [1, 13, 1], [2, 20, 9]]]

In [15]:
# creating sample random week with 5 days and 11 employees
random_staff_planning = generate_random_staff_planning(5, 11)

# measuring the cost of this randomly generated week compared to the hourly staff needed
cost(staffplanning_to_hourlyplanning(random_staff_planning), hourlystaff_needed)

211

# Creating the Genetic Algorithm
## Create Generation
Here, I'm writing a function to create the initial generation of parents with random planning

In [16]:
# create a parent generation of n parents
def create_parent_generation(n_parents, n_days = 7, n_staff = 11):
    parents = []
    for i in range(n_parents):
        parent = generate_random_staff_planning(n_days = n_days, n_staff = n_staff)
        parents.append(parent)
    return parents

In [17]:
# creating a parent generation with 10 parents
parent_generation = create_parent_generation(10, n_days = 5)

# showing one of the parents
parent_generation[0]

[[[0, 1, 7],
  [1, 4, 8],
  [2, 8, 1],
  [3, 2, 4],
  [4, 21, 2],
  [5, 2, 1],
  [6, 14, 2],
  [7, 22, 0],
  [8, 10, 7],
  [9, 15, 7],
  [10, 17, 1]],
 [[0, 20, 5],
  [1, 14, 6],
  [2, 20, 8],
  [3, 16, 5],
  [4, 22, 5],
  [5, 19, 3],
  [6, 14, 4],
  [7, 12, 3],
  [8, 22, 8],
  [9, 12, 5],
  [10, 20, 3]],
 [[0, 17, 2],
  [1, 9, 2],
  [2, 19, 1],
  [3, 7, 6],
  [4, 10, 8],
  [5, 21, 3],
  [6, 10, 0],
  [7, 11, 4],
  [8, 5, 3],
  [9, 4, 0],
  [10, 10, 6]],
 [[0, 15, 1],
  [1, 0, 5],
  [2, 7, 9],
  [3, 12, 9],
  [4, 15, 6],
  [5, 11, 5],
  [6, 15, 9],
  [7, 16, 4],
  [8, 13, 5],
  [9, 3, 3],
  [10, 12, 4]],
 [[0, 5, 5],
  [1, 0, 3],
  [2, 4, 4],
  [3, 14, 5],
  [4, 11, 9],
  [5, 2, 8],
  [6, 19, 9],
  [7, 14, 8],
  [8, 19, 5],
  [9, 18, 7],
  [10, 9, 0]]]

## Create Cross Over / Combination
There are two key steps in Genetic Algorihtms: mating and mutation. In the mating step, the new generation is formed through natural selection from the offspring of individuals of the parent population. In this function, I'm randomly combining the genes of two parents to create the offspring.

In [18]:
# for each iteration, randomly select two parents and make a random combination of those two parents
# by applying a randomly generated yes/no mask to the two selected parents
def random_combine(parents, n_offspring):
    # get the number of parents
    n_parents = len(parents)
    
    # get the number of days
    n_periods = len(parents[0])
    
    # get the number of employees working each day
    n_employees = len(parents[0][0])
    
    offspring = []
    
    for i in range(n_offspring):
        # randomly select the dad to be any of the parents
        # in np.random.randint, the highest value is one below the high value, so I'll make high = n_parents
        random_dad = parents[np.random.randint(low=0, high=n_parents)]
        random_mom = parents[np.random.randint(low=0, high=n_parents)]
        
        # create a mask that randomly selects 0's and 1's in the shape of the random_dad
        dad_mask = np.random.randint(low = 0, high= 2, size = np.array(random_dad).shape)
        # create a mom_mask which is the exact opposite of the dad mask. Therefore it only chooses from the mom or the dad
        mom_mask = np.logical_not(dad_mask)
        
        # np.add and np.multiply work element-wise
        # I multiply the dad with the dad_mask. Since the dad_mask contains only 0's and 1's, this will result in only the 
        # chosen dad traits. The same with the mom. Then add them together to get the combination of mom and dad traits
        child = np.add(np.multiply(random_dad, dad_mask), np.multiply(random_mom, mom_mask))
        
        offspring.append(child)
    return offspring

In [19]:
# this creates two offspring from the parent_generation
random_combine(parent_generation, 2)

[array([[[ 0, 19,  4],
         [ 1,  4,  1],
         [ 2,  8,  1],
         [ 3,  2,  2],
         [ 4, 11,  2],
         [ 5,  2,  1],
         [ 6, 13,  4],
         [ 7, 22,  8],
         [ 8, 10,  4],
         [ 9, 15,  7],
         [10, 13,  1]],
 
        [[ 0, 20,  0],
         [ 1, 14,  6],
         [ 2, 20,  8],
         [ 3,  9,  8],
         [ 4, 18,  5],
         [ 5, 19,  3],
         [ 6,  3,  4],
         [ 7, 13,  3],
         [ 8, 22,  3],
         [ 9, 11,  5],
         [10,  1,  3]],
 
        [[ 0, 17,  2],
         [ 1,  9,  2],
         [ 2, 19,  1],
         [ 3,  7,  4],
         [ 4, 10,  2],
         [ 5, 21,  8],
         [ 6, 10,  0],
         [ 7, 18,  0],
         [ 8, 12,  3],
         [ 9,  4,  6],
         [10, 18,  6]],
 
        [[ 0, 15,  1],
         [ 1,  0,  5],
         [ 2,  1,  9],
         [ 3,  6,  5],
         [ 4, 15,  6],
         [ 5,  5,  5],
         [ 6,  0,  9],
         [ 7, 16,  5],
         [ 8, 13,  6],
         [ 9,  3,  3],
  

## Mutation
Mutation consists of adding a completely random change to the new generation. This can allow new genes to be brought into the population that aren't currently present. For example, if a certain gene has been bred out after a few iterations due to randomness and natural selection, without mutation, that gene could never be re-introduced, even if it might actually provide a better solution later on.

In [20]:
# mutates one parent
def mutate_parent(parent, n_mutations):
    # number of days
    size1 = parent.shape[0]
    # number of employees
    size2 = parent.shape[1]
    
    for i in range(n_mutations):
        # randomly select one of the days
        rand1 = np.random.randint(0, size1)
        # randomly select one of the employees
        rand2 = np.random.randint(0, size2)
        # randomly select either the start time or the hours worked
        rand3 = np.random.randint(1, 3)
        
        # select one random employee on a random day and modify either the start time / hours worked to be a number between 0-10
        # do this for however many mutations were specified
        if (rand3 == 1):
            # if rand3 == 1 that means I am mutating the start time, the start time can be any number from 0-23
            parent[rand1,rand2,rand3] = np.random.randint(0,24)
        else:
            # if rand3 == 2 that means I am mutating the hours worked, the hours worked can be any number from 0-9
            parent[rand1,rand2,rand3] = np.random.randint(0,10)
            
    return parent

# mutates an entire generation
def mutate_gen(parent_gen, n_mutations):
    mutated_parent_gen = []
    for parent in parent_gen:
        # for every parent, mutate as many of the genes as specified
        mutated_parent_gen.append(mutate_parent(parent, n_mutations))
    return mutated_parent_gen

## Selection Feasibility
This function ensures that only the viable parents are selected

In [21]:
# if an employee works for more than 10 hours, then it is not acceptable
def is_acceptable(parent):
    return np.logical_not((np.array(parent)[:,:,2:]>10).any())

# select only the parents that are acceptable in the parent generation
def select_acceptable(parent_gen):
    parent_gen = [parent for parent in parent_gen if is_acceptable(parent)]
    return parent_gen

## Selection Cost (inverse fitness)
This function selects the n_best number of parents from a generation based on my cost function

In [22]:
def select_best(parent_gen, hourlystaff_needed, n_best):
    costs = []
    for idx, parent_staff_planning in enumerate(parent_gen):
        # convert each of the parents to the hourlyplanning
        parent_hourly_planning = staffplanning_to_hourlyplanning(parent_staff_planning)
        # calculate the cost of the parent hourly planning with the staff needed
        parent_cost = cost(parent_hourly_planning, hourlystaff_needed)
        # append the cost to the list of costs
        costs.append([idx, parent_cost])
    # find and print the max and min cost
    print('generations best is: {}, generations worst is: {}'.format(pd.DataFrame(costs)[1].min(), pd.DataFrame(costs)[1].max()))    
    
    # sort in ascending value by the parent cost
    costs_tmp = pd.DataFrame(costs).sort_values(by = 1, ascending = True).reset_index(drop=True)
    
    # select only the top n_best parents
    selected_parents_idx = list(costs_tmp.iloc[:n_best,0])
    
    # create list containing tuples of the (idx, parent) out of the selected parents
    selected_parents = [parent for idx, parent in enumerate(parent_gen) if idx in selected_parents_idx]
    
    return selected_parents

In [23]:
selected_parents = select_best(parent_generation, hourlystaff_needed, 2)

generations best is: 202, generations worst is: 243


In [24]:
np.array(selected_parents).shape

(2, 5, 11, 3)

# Putting all of the functions together
This function puts everything together into a function that can be run in order to find the best staff planning solution.

In [25]:
def gen_algo(hourlystaff_needed, n_iterations):
    # set up the generation size
    generation_size = 500
    
    # create the parent generation
    parent_gen = create_parent_generation(n_parents = generation_size, n_days = 5, n_staff = 11)
    for it in range(n_iterations):
        # select only the acceptable parents
        parent_gen = select_acceptable(parent_gen)
        # select the top 100 parents to "breed"
        parent_gen = select_best(parent_gen, hourlystaff_needed, n_best = 100)
        # combine the parents to make generation_size number of children
        parent_gen = random_combine(parent_gen, n_offspring = generation_size)
        # create some random mutations
        parent_gen = mutate_gen(parent_gen, n_mutations = 1)
    best_child = select_best(parent_gen, hourlystaff_needed, n_best = 1)
    return best_child

# Execute all

In [26]:
best_planning = gen_algo(hourlystaff_needed, n_iterations = 100)

generations best is: 168, generations worst is: 268
generations best is: 165, generations worst is: 240
generations best is: 156, generations worst is: 228
generations best is: 144, generations worst is: 220
generations best is: 147, generations worst is: 210
generations best is: 138, generations worst is: 205
generations best is: 139, generations worst is: 200
generations best is: 118, generations worst is: 207
generations best is: 117, generations worst is: 188
generations best is: 119, generations worst is: 187
generations best is: 120, generations worst is: 177
generations best is: 99, generations worst is: 170
generations best is: 110, generations worst is: 177
generations best is: 108, generations worst is: 167
generations best is: 101, generations worst is: 164
generations best is: 100, generations worst is: 164
generations best is: 97, generations worst is: 180
generations best is: 92, generations worst is: 163
generations best is: 91, generations worst is: 149
generations best

In [27]:
print(best_planning)

[array([[[ 0, 12,  2],
        [ 1, 16,  4],
        [ 2,  6,  8],
        [ 3, 12,  8],
        [ 4,  5,  4],
        [ 5, 12,  8],
        [ 6, 12,  2],
        [ 7,  6,  8],
        [ 8, 17,  8],
        [ 9,  6,  3],
        [10, 17,  8]],

       [[ 0, 17,  3],
        [ 1, 12,  2],
        [ 2,  6,  3],
        [ 3,  6,  8],
        [ 4,  6,  3],
        [ 5, 17,  9],
        [ 6, 17,  7],
        [ 7, 12,  8],
        [ 8, 17,  3],
        [ 9, 12,  8],
        [10,  6,  8]],

       [[ 0, 17,  3],
        [ 1, 20,  0],
        [ 2,  6,  3],
        [ 3,  6,  3],
        [ 4,  6,  8],
        [ 5, 12,  8],
        [ 6, 17,  3],
        [ 7, 12,  8],
        [ 8, 17,  8],
        [ 9,  6,  8],
        [10, 17,  7]],

       [[ 0, 17,  7],
        [ 1,  6,  3],
        [ 2, 17,  3],
        [ 3, 12,  8],
        [ 4, 12,  8],
        [ 5,  6,  8],
        [ 6, 17,  3],
        [ 7,  6,  3],
        [ 8, 17,  7],
        [ 9,  6,  8],
        [10, 12,  2]],

       [[ 0,  6,  3],
 

# Comparison between the best child's planning and the hourly staff needed

In [28]:
print(staffplanning_to_hourlyplanning(best_planning[0]))

[[0 0 0 0 0 1 4 4 4 2 2 2 6 6 2 2 3 5 5 5 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 5 5 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 4 4 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 5 5 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 5 5 2 2 3 6 6 6 2 2 2 2]]


In [29]:
print(hourlystaff_needed)

[[0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]
 [0 0 0 0 0 0 4 4 4 2 2 2 6 6 2 2 2 6 6 6 2 2 2 2]]
